In [76]:
import xmltodict
import cv2
import os
import zipfile
import shutil

In [80]:
def extract_annotations(path_to_zip_file):
    """Extract the zip file on the given path to a directory under the same name."""
    path_to_zip_file_list = path_to_zip_file.split('/')
    path_to_zip_len = len(path_to_zip_file_list)
    path_to_extracted_file = '/' + os.path.join(*path_to_zip_file_list[:path_to_zip_len - 1])
    destination_dir_name = path_to_zip_file_list[path_to_zip_len - 1].split('.')[0]
    zip_ref = zipfile.ZipFile(path_to_zip_file, 'r')
    zip_ref.extractall(path_to_extracted_file)
    zip_ref.close()
    # Remove an unnecessary automatically created directory.
    shutil.rmtree(os.path.join(path_to_extracted_file, "__MACOSX"))
    return os.path.join(path_to_extracted_file, destination_dir_name)

PATH_TO_ZIP_FILE = "/Users/alejandrosanchezaristizabal/Desktop/annotations_2.zip"
annotations_path = extract_annotations(PATH_TO_ZIP_FILE)
print(annotations_path)

/Users/alejandrosanchezaristizabal/Desktop/annotations_2


In [102]:
def get_images_path(annotations_path):
    """"Get the path to the image files."""
    path_to_annotations_list = annotations_path.split('/')
    path_to_annotations_len = len(path_to_annotations_list)
    path_to_images = '/' + os.path.join(*path_to_annotations_list[:path_to_annotations_len - 1])
    return os.path.join(path_to_images, "images")

ANNOTATIONS_PATH = annotations_path
images_path = get_images_path(ANNOTATIONS_PATH)

/Users/alejandrosanchezaristizabal/Desktop/images


In [103]:
def get_annotation_file_paths(annotation_path):
    """Get a list with the XML file paths."""
    xml_file_paths = list()
    for subdir, dirs, files in os.walk(annotations_path):
        for file in files:
            file_path = os.path.join(subdir, file)
            if file_path.endswith(".xml"):
                xml_file_paths.append(file_path)
    return xml_file_paths

xml_file_paths = get_annotation_file_paths(ANNOTATIONS_PATH)
print(xml_file_paths)

['/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/12.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/13.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/10.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/4.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/5.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/7.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/2.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/3.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/1.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/1/0.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/4/14.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/4/15.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/4/17.xml', '/Users/alejandrosanchezaristizabal/Desktop/annotations_2/4/16.xml', '/Users/alejandrosanchezaristizabal/Desk

In [105]:
def get_info_from_object_item(object_item, info):
    """Updates the given dictionary using the values of the given object item."""
    label = object_item["name"]
    bnd_box = object_item["bndbox"]
    x_min = int(bnd_box["xmin"])
    y_min = int(bnd_box["ymin"])
    x_max = int(bnd_box["xmax"])
    y_max = int(bnd_box["ymax"])
    info["label"] = label
    info["xmin"] = x_min
    info["ymin"] = y_min
    info["xmax"] = x_max
    info["ymax"] = y_max
    return info

def parse_xml(document):
    """Parse the given document in order to extract the relevant values."""
    folder = document["annotation"]["folder"]
    filename = document["annotation"]["filename"]
    infos = list()
    object_items = document["annotation"]["object"]
    if isinstance(object_items, list):
        for object_item in object_items:
            info = dict()
            info["folder"] = folder
            info["filename"] = filename
            info = get_info_from_object_item(object_item, info)
            infos.append(info)
    else:
        info = dict()
        info["folder"] = folder
        info["filename"] = filename
        info = get_info_from_object_item(object_items, info)
        infos.append(info)
    return infos

def get_info_from_xml_files(xml_file_paths):
    """Get the labelling information within each XML file."""
    info_from_xml_files = list()
    for xml_file_path in xml_file_paths:
        with open(xml_file_path) as fd:
            document = xmltodict.parse(fd.read())
            for info in parse_xml(document):
                info_from_xml_files.append(info)
    return info_from_xml_files

XML_FILE_PATHS = xml_file_paths
info_from_xml_files = get_info_from_xml_files(XML_FILE_PATHS)

In [106]:
# TODO.
# Function for cropping image and returning it and the corresponding label.
# Function for generating a unique name to the cropped image and assigning it to 
# train/valid/test sets and returning the image, label, name and set.
# Function for saving the image and appending a new row to the metadata file.